In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
from src.web.connect_openrouter import get_response_llm

In [4]:
get_response_llm('Как тебя зовут?')

'Меня зовут MiniMax-M2, я помощник ИИ, созданный компанией MiniMax. Чем могу помочь?'

In [5]:
from typing import TypedDict, List, Dict

# LangGraph core
from langgraph.graph import StateGraph, START, END

# Используем уже готовое подключение к OpenRouter
from src.web.connect_openrouter import get_response_llm


class AgentState(TypedDict):
    """Состояние графа с историей сообщений в формате ChatML-подобного списка."""

    messages: List[Dict[str, str]]


def _call_model(state: AgentState) -> AgentState:
    """Узел-главная логика: берём последний запрос пользователя и получаем ответ LLM."""

    user_message = state["messages"][-1]["content"] if state["messages"] else ""
    answer = get_response_llm(user_message)
    updated_messages = state["messages"] + [{"role": "assistant", "content": answer}]
    return {"messages": updated_messages}


def create_simple_agent():
    """Создаёт и компилирует простой агент на LangGraph с одним узлом модельного вызова."""

    graph = StateGraph(AgentState)
    graph.add_node("model", _call_model)
    graph.add_edge(START, "model")
    graph.add_edge("model", END)
    return graph.compile()


def run_simple_agent(query: str) -> str:
    """Утилита для быстрого запуска агента одной командой.

    Возвращает текст ответа ассистента.
    """

    app = create_simple_agent()
    initial_state: AgentState = {"messages": [{"role": "user", "content": query}]}
    final_state: AgentState = app.invoke(initial_state)
    # Последнее сообщение — ответ ассистента
    return final_state["messages"][-1]["content"]


if __name__ == "__main__":
    # Пример локального запуска: python -m src.web.agents.simple_agent
    demo_answer = run_simple_agent("Привет! Расскажи, что ты умеешь?")
    print(demo_answer)




Привет! Я — MiniMax-M2, ваш универсальный AI-помощник! Вот что я умею:

🔍 **Исследования и анализ**
- Поиск и анализ информации в интернете
- Обработка документов и данных
- Научные исследования

✍️ **Творчество и письмо**
- Создание текстов любого жанра
- Копирайтинг и редактура
- Переводы между языками
- Поэзия и креативное письмо

💻 **Техническая помощь**
- Программирование (Python, JavaScript, и др.)
- Веб-разработка
- Работа с данными
- Решение технических задач

🎓 **Образование и обучение**
- Объяснение сложных концепций
- Помощь с домашними заданиями
- Языковое обучение
- Подготовка к экзаменам

💼 **Бизнес-решения**
- Планирование и стратегия
- Анализ рынка
- Создание презентаций
- Финансовые расчеты

🎯 **Повседневная помощь**
- Планирование мероприятий
- Генерация идей
- Решение бытовых вопросов
- Развлечения и игры

Просто задайте мне вопрос или опишите задачу — я постараюсь помочь! Что вас интересует? 😊


In [12]:
from src.web.agents.md_scene_agent import run_md_scene_agent

In [39]:
res2 = run_md_scene_agent( md_path = '/Users/egorbykov/Desktop/Работа/2025/hackatons/wink/data/parsing/md/1 кейс Сценарии c Таблицами/Отмороженные 1 (c таблицей)/ОТМОРОЖЕННЫЕ 1 - 01 Пацаны, у нас проблема (для ИРИ 28.11)(изменения).md', sample_chars = 6000)

In [42]:
res2

[{'id': '1-1',
  'title': 'ИНТ. КВАРТИРА ФИЛА (прихожая) – НОЧЬ',
  'text': '1-1.\tИНТ. КВАРТИРА ФИЛА (прихожая) – НОЧЬ\n\nМы крупно видим губы у телефонной трубки, пальцы держат обручальное кольцо, висящее на цепочке. Отвечая по телефону, Фил покусывает кольцо, как это бывает с теми, кто носит на шее крестик.\n\nФИЛ\n\nА до завтра ~~это подождать не может?~~ оно не ждёт?\n\n(слушает)\n\nЯ только ~~недавно~~ пацанов по домам отпустил, ~~мы~~ весь день ~~по должникам мотались~~ фраеров бомбили.\n\n(слушает)\n\n~~Хорошо~~ Лады. Скоро будем.\n\nПока он это говорит, камера отъезжает, и мы видим ФИЛА, 27 лет, суровое, но располагающее лицо, глаза смотрят уверенно, голос мягкий, но властный.\n\nМы видим, что сзади в коридоре, слушая разговор, стоит девушка лет 20 (ДАША). Даже в домашней одежде, без макияжа она красавица. На руках у неё годовалый ребёнок, которого она укачивает, он спит.\n\nФил вешает трубку и разворачивается, увидев, что она слышала, он грустнеет. Убирает кольцо на цепочке п

In [43]:
# Сохранение в файл
import json
with open('../../data/json/ОТМОРОЖЕННЫЕ 1 - 01 Пацаны, у нас проблема (для ИРИ 28.11)(изменения).json', 'w', encoding='utf-8') as f:
    json.dump(res2, f, ensure_ascii=False, indent=2)

In [73]:
from src.web.agents.scene_insights_agent import run_scene_insights_agent

In [74]:
import json as json_module

In [75]:
with open('../../data/json/ОТМОРОЖЕННЫЕ 1 - 01 Пацаны, у нас проблема (для ИРИ 28.11)(изменения).json', "r", encoding="utf-8") as fh:
    scenes_input = json_module.load(fh)

In [76]:
fields = [
    "Серия",
    "Сцена",
    "Режим",
    "Инт / нат",
    "Объект",
    "Подобъект",
    "Синопсис",
    "Персонажи",
    "Массовка / Групповка",
    "Грим / Костюм",
    "Реквизит / Животное / Игровые фото",
    "Игровой транспорт",
    "Декорация",
    "Примечание / Спец. оборудование / Администрация / CG",
    "Трюк / Пиротехник"
]

desc_fields = [
    "Пример: 1, 2, 3, 4",
    "Пример: 1-1, 1-2, 1-3, 1-4, первое число ",
    "Пример: НОЧЬ, ДЕНЬ, УТРО, ВЕЧЕР",
    "Инт - это интерер / нат - это экстерер",
    "Нечто более крупное. Пример: квартира, город, офис",
    "Более точно где в объекте. прихожая, конференц-зал, спортзал, машина",
    "краткое описнание",
    "лица участвующие в сцене",
    "",
    "только для персонажей",
    "",
    "",
    "",
    "",
    ""
]

In [77]:
dataframe = run_scene_insights_agent(scenes_input[:4], fields, desc_fields)

In [79]:
dataframe.to_excel('test.xlsx', index=False)